In [1]:
# ============================================================
# 03_dispatch_arbitrage.ipynb
# Despacho óptimo (arbitraje) por escenario y capacidad
# P = 1 MW, E = {1,2,4} MWh, dt = 1h, SOC0 = SOC24 = 0
# Salidas:
#  - data/processed/dispatch_<scenario>_E<cap>.csv
#  - data/processed/dispatch_summary.csv
#  - results/figures/*.png (opcional pero recomendable)
# ============================================================

# -------- 0) Montar Google Drive --------
from google.colab import drive
drive.mount('/content/drive')

# -------- 1) Dependencias --------
!pip -q install pandas numpy scipy matplotlib

import os
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import matplotlib.pyplot as plt

# -------- 2) Rutas --------
BASE_PATH = "/content/drive/MyDrive/energy_storage_esios"
assert os.path.exists(BASE_PATH), "BASE_PATH no existe. ¿Drive montado?"

SCEN_PATH = os.path.join(BASE_PATH, "data", "processed", "scenarios_qstd_2024.csv")
assert os.path.exists(SCEN_PATH), f"No existe: {SCEN_PATH}"

OUT_DATA_DIR = os.path.join(BASE_PATH, "data", "processed")
OUT_FIG_DIR = os.path.join(BASE_PATH, "results", "figures")
os.makedirs(OUT_DATA_DIR, exist_ok=True)
os.makedirs(OUT_FIG_DIR, exist_ok=True)

print("Escenarios:", SCEN_PATH)
print("Out data:", OUT_DATA_DIR)
print("Out figs:", OUT_FIG_DIR)

# -------- 3) Parámetros del almacenamiento --------
P_MW = 1.0
CAPACITIES_MWH = [1.0, 2.0, 4.0]   # E
DT_H = 1.0                         # 1 hora

# -------- 4) Cargar escenarios --------
scen = pd.read_csv(SCEN_PATH)
hour_cols = [f"h{h:02d}" for h in range(24)]
assert all(c in scen.columns for c in hour_cols), "Faltan columnas horarias h00..h23"

scen["scenario_name"] = scen.apply(lambda r: f"Q{int(r['std_quartile'])}_{r['date_local']}", axis=1)

print("\nEscenarios disponibles:")
print(scen[["scenario_name","date_local","std_quartile","mean_daily","std_daily"]])

# -------- 5) Optimizador LP --------
def optimize_dispatch_lp(prices_eur_mwh: np.ndarray, P: float, E: float, dt: float):
    """
    Decisiones por hora t=0..23:
      c_t >= 0 (MW)  carga
      d_t >= 0 (MW)  descarga
      soc_t (MWh)    estado al inicio de la hora t
    Dinámica:
      soc_{t+1} = soc_t + (c_t - d_t)*dt
    Restricciones:
      0 <= c_t <= P
      0 <= d_t <= P
      0 <= soc_t <= E
      soc_0 = 0, soc_24 = 0
    Objetivo:
      max sum_t price_t * (d_t - c_t) * dt
    """

    T = 24
    # Variables: [c_0..c_23, d_0..d_23, soc_0..soc_24]
    n_c = T
    n_d = T
    n_soc = T + 1
    n_vars = n_c + n_d + n_soc

    def idx_c(t): return t
    def idx_d(t): return n_c + t
    def idx_soc(t): return n_c + n_d + t  # t=0..24

    # Minimización => minimizar -beneficio
    c_obj = np.zeros(n_vars)
    for t in range(T):
        c_obj[idx_c(t)] = +prices_eur_mwh[t] * dt   # coste por cargar
        c_obj[idx_d(t)] = -prices_eur_mwh[t] * dt   # ingreso por descargar

    # Igualdades: dinámica SOC (24 eq) + soc_0 = 0 + soc_24 = 0
    A_eq = []
    b_eq = []

    # soc_{t+1} - soc_t - (c_t - d_t)*dt = 0
    for t in range(T):
        row = np.zeros(n_vars)
        row[idx_soc(t+1)] = 1.0
        row[idx_soc(t)] = -1.0
        row[idx_c(t)] = -dt
        row[idx_d(t)] = +dt
        A_eq.append(row)
        b_eq.append(0.0)

    # soc_0 = 0
    row0 = np.zeros(n_vars)
    row0[idx_soc(0)] = 1.0
    A_eq.append(row0); b_eq.append(0.0)

    # soc_24 = 0
    row24 = np.zeros(n_vars)
    row24[idx_soc(T)] = 1.0
    A_eq.append(row24); b_eq.append(0.0)

    A_eq = np.vstack(A_eq)
    b_eq = np.array(b_eq)

    # Bounds
    bounds = []
    # c_t
    bounds += [(0.0, P) for _ in range(T)]
    # d_t
    bounds += [(0.0, P) for _ in range(T)]
    # soc_t
    bounds += [(0.0, E) for _ in range(T+1)]

    res = linprog(
        c=c_obj,
        A_eq=A_eq,
        b_eq=b_eq,
        bounds=bounds,
        method="highs"
    )

    if not res.success:
        raise RuntimeError(f"LP no convergió: {res.message}")

    x = res.x
    c = x[0:T]
    d = x[T:2*T]
    soc = x[2*T:2*T + (T+1)]

    net = d - c  # MW (positivo descarga, negativo carga)
    profit = float(np.sum(prices_eur_mwh * net * dt))  # EUR (porque €/MWh * MW*h)

    return c, d, net, soc, profit

def classify_action(net_mw: float, eps: float = 1e-6) -> str:
    if net_mw > eps:
        return "D"
    if net_mw < -eps:
        return "C"
    return "I"

# -------- 6) Ejecutar optimización por escenario y capacidad --------
summary_rows = []

for _, r in scen.iterrows():
    scenario_name = r["scenario_name"]
    prices = r[hour_cols].to_numpy(dtype=float)

    for E in CAPACITIES_MWH:
        c, d, net, soc, profit = optimize_dispatch_lp(prices, P=P_MW, E=E, dt=DT_H)

        # Tabla de resultados por hora
        hours = np.arange(24)
        df_out = pd.DataFrame({
            "scenario": scenario_name,
            "date_local": r["date_local"],
            "std_quartile": int(r["std_quartile"]),
            "hour": hours,
            "price_eur_mwh": prices,
            "charge_mw": c,
            "discharge_mw": d,
            "net_mw": net,
            "action": [classify_action(v) for v in net],
            "soc_start_mwh": soc[:-1],
            "soc_end_mwh": soc[1:],
            "capacity_mwh": E,
            "power_mw": P_MW,
        })

        # Guardar CSV por caso
        safe_name = scenario_name.replace(":", "_").replace("/", "_")
        out_csv = os.path.join(OUT_DATA_DIR, f"dispatch_{safe_name}_E{int(E)}MWh.csv")
        df_out.to_csv(out_csv, index=False)

        # Resumen
        summary_rows.append({
            "scenario": scenario_name,
            "date_local": r["date_local"],
            "std_quartile": int(r["std_quartile"]),
            "capacity_mwh": E,
            "power_mw": P_MW,
            "profit_eur_per_day": profit,
            "energy_charged_mwh": float(np.sum(c)*DT_H),
            "energy_discharged_mwh": float(np.sum(d)*DT_H),
            "max_soc_mwh": float(np.max(soc)),
        })

        # Figura rápida (precio + SOC) — recomendable para control
        fig_path = os.path.join(OUT_FIG_DIR, f"dispatch_{safe_name}_E{int(E)}MWh.png")
        plt.figure()
        plt.plot(hours, prices, label="Precio (€/MWh)")
        plt.xlabel("Hora (local)")
        plt.ylabel("Precio (€/MWh)")
        plt.title(f"{scenario_name} | E={int(E)} MWh, P={int(P_MW)} MW")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(fig_path, dpi=150)
        plt.close()

        fig_path2 = os.path.join(OUT_FIG_DIR, f"soc_{safe_name}_E{int(E)}MWh.png")
        plt.figure()
        plt.step(np.arange(25), soc, where="post", label="SOC (MWh)")
        plt.xlabel("Hora (local, bordes 0..24)")
        plt.ylabel("SOC (MWh)")
        plt.title(f"SOC | {scenario_name} | E={int(E)} MWh, P={int(P_MW)} MW")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(fig_path2, dpi=150)
        plt.close()

print("\nListo: CSVs de despacho y figuras generadas.")

# -------- 7) Guardar resumen --------
summary = pd.DataFrame(summary_rows).sort_values(["std_quartile","capacity_mwh"]).reset_index(drop=True)
summary_path = os.path.join(OUT_DATA_DIR, "dispatch_summary.csv")
summary.to_csv(summary_path, index=False)

print("\nResumen guardado:", summary_path)
summary


Mounted at /content/drive
Escenarios: /content/drive/MyDrive/energy_storage_esios/data/processed/scenarios_qstd_2024.csv
Out data: /content/drive/MyDrive/energy_storage_esios/data/processed
Out figs: /content/drive/MyDrive/energy_storage_esios/results/figures

Escenarios disponibles:
   scenario_name  date_local  std_quartile  mean_daily  std_daily
0  Q1_2024-03-07  2024-03-07             1    8.522917   9.676325
1  Q2_2024-07-31  2024-07-31             2  105.341250  17.782959
2  Q3_2024-04-28  2024-04-28             3   28.722500  25.055756
3  Q4_2024-10-13  2024-10-13             4   52.750417  36.788800

Listo: CSVs de despacho y figuras generadas.

Resumen guardado: /content/drive/MyDrive/energy_storage_esios/data/processed/dispatch_summary.csv


,scenario,date_local,std_quartile,capacity_mwh,power_mw,profit_eur_per_day,energy_charged_mwh,energy_discharged_mwh,max_soc_mwh
0,Q1_2024-03-07,2024-03-07,1,1.0,1.0,48.37,3.0,3.0,1.0
1,Q1_2024-03-07,2024-03-07,1,2.0,1.0,88.74,5.0,5.0,2.0
2,Q1_2024-03-07,2024-03-07,1,4.0,1.0,132.10,9.0,9.0,4.0
3,Q2_2024-07-31,2024-07-31,2,1.0,1.0,70.23,3.0,3.0,1.0
4,Q2_2024-07-31,2024-07-31,2,2.0,1.0,126.03,5.0,5.0,2.0
5,Q2_2024-07-31,2024-07-31,2,4.0,1.0,202.61,8.0,8.0,4.0
6,Q3_2024-04-28,2024-04-28,3,1.0,1.0,80.93,4.0,4.0,1.0
7,Q3_2024-04-28,2024-04-28,3,2.0,1.0,153.89,6.0,6.0,2.0
8,Q3_2024-04-28,2024-04-28,3,4.0,1.0,273.42,7.0,7.0,4.0
9,Q4_2024-10-13,2024-10-13,4,1.0,1.0,138.71,5.0,5.0,1.0
